In [49]:
# Importing necessary libraries
import torch
import torchtext
from torchtext.vocab import GloVe
import torch.nn.functional as F
import torch.nn as nn
import torch

class CNNClassifier(nn.Module):
    def __init__(self, embedding_vectors, k1=2, k2=4, n1=4, n2=16):
        super(CNNClassifier, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding.from_pretrained(embedding_vectors)
        embedding_dim = embedding_vectors.size(1)

        # Convolutional layers
        # (in_channels, out_channels, kernel_size)
        self.conv1 = nn.Conv2d(1, n1, (k1, embedding_dim), bias=False)
        self.conv2 = nn.Conv2d(1, n2, (k2, embedding_dim), bias=False)

        # Fully connected layer
        self.fc = nn.Linear(n1 + n2, 1)

    def forward(self, x):
        # Embedding
        x = self.embedding(x)
        x = x.transpose(0, 1)

        # Add an extra dimension for the channel
        x = x.unsqueeze(1)

        # Apply Convolutional layers and ReLU activation
        x1 = F.relu(self.conv1(x)).squeeze(3)
        x2 = F.relu(self.conv2(x)).squeeze(3)

        # Max pooling over the sentence length
        x1 = F.max_pool1d(x1, x1.size(2)).squeeze(2)
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2)

        # Concatenate the outputs
        x = torch.cat((x1, x2), 1)

        # Pass through the fully connected layer
        x = self.fc(x).squeeze(1)
        
        return x
    
class BaselineModel(nn.Module):
    def __init__(self, embedding_vectors):
        super(BaselineModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_vectors)
        self.fc = nn.Linear(embedding_vectors.size(1), 1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.mean(dim=0)
        x = self.fc(x).squeeze(1)
        return x

In [64]:
# Load the GloVe embeddings
glove = torchtext.vocab.GloVe(name="6B",dim=100)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_baseline = BaselineModel(glove.vectors).to(device)
model_baseline.load_state_dict(torch.load("model_baseline.pt"))

model_CNN = CNNClassifier(glove.vectors, k1=2, n1=4, k2=4, n2=16).to(device)
model_CNN.load_state_dict(torch.load("model_CNN.pt"))

model_baseline.eval()  # setting the model to evaluation mode
model_CNN.eval()  # setting the model to evaluation mode

model_baseline, model_CNN, glove

(BaselineModel(
   (embedding): Embedding(400000, 100)
   (fc): Linear(in_features=100, out_features=1, bias=True)
 ),
 CNNClassifier(
   (embedding): Embedding(400000, 100)
   (conv1): Conv2d(1, 4, kernel_size=(2, 100), stride=(1, 1), bias=False)
   (conv2): Conv2d(1, 16, kernel_size=(4, 100), stride=(1, 1), bias=False)
   (fc): Linear(in_features=20, out_features=1, bias=True)
 ),
 <torchtext.vocab.vectors.GloVe at 0x2bdfcb640>)

In [65]:
def process_input(sentence):
    tokens = sentence.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)
    
    return token_tensor

In [74]:
def predict(sentence, model_choice):
    """
    Predict the classification of the sentence using the selected model.

    Parameters:
    - sentence (str): The input sentence for prediction.
    - model_choice (str): The selected model ("Baseline" or "CNN").

    Returns:
    - Prediction result as a string.
    """

    # Process the input sentence to get the tensor
    token_tensor = process_input(sentence)
    
    if model_choice == "Baseline":
        # Get predictions from the baseline model
        output = model_baseline(token_tensor)
        print(output)
    else:
        # Get predictions from the CNN model
        output = model_CNN(token_tensor)
        
    prob = torch.sigmoid(output)  # Using sigmoid activation instead of softmax
    predicted_class = 'subjective' if prob[0] > 0.5 else 'objective'
    
    return f"Class: {predicted_class}, Probability: {prob[0].item():.4f}"


In [75]:
predict("his life is a whirl of seedy bars , seedier girlfriends and a wife two towns away .", "Baseline")

tensor([-3.3759], grad_fn=<SqueezeBackward1>)


'Class: objective, Probability: 0.0331'

In [76]:
import gradio as gr

gApp = gr.Interface(fn=predict, 
                        inputs=[gr.Textbox(label="type a sentence here"), 
                                gr.Radio(["Baseline", "CNN"], label="Model Choice")], 
                        outputs=gr.Textbox(label="Model's Prediction"))

gApp.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


tensor([3.5610], grad_fn=<SqueezeBackward1>)
tensor([-3.6983], grad_fn=<SqueezeBackward1>)
tensor([3.5610], grad_fn=<SqueezeBackward1>)
tensor([3.5610], grad_fn=<SqueezeBackward1>)
tensor([3.6626], grad_fn=<SqueezeBackward1>)
tensor([6.8853], grad_fn=<SqueezeBackward1>)
tensor([1.8826], grad_fn=<SqueezeBackward1>)
tensor([1.8826], grad_fn=<SqueezeBackward1>)
tensor([-1.5518], grad_fn=<SqueezeBackward1>)
tensor([0.5581], grad_fn=<SqueezeBackward1>)
tensor([-1.5518], grad_fn=<SqueezeBackward1>)
tensor([-4.8716], grad_fn=<SqueezeBackward1>)
tensor([7.9978], grad_fn=<SqueezeBackward1>)
tensor([7.5710], grad_fn=<SqueezeBackward1>)
tensor([7.5710], grad_fn=<SqueezeBackward1>)
tensor([6.6265], grad_fn=<SqueezeBackward1>)
tensor([1.2875], grad_fn=<SqueezeBackward1>)
tensor([1.2875], grad_fn=<SqueezeBackward1>)


## Run each cell one by one